# CAPSTONE PROJECT: BATTLE OF THE NEIGHBORHOODS (PART 2)


### Applied Data science Capstone by Vishaal Rao

## Introduction: Business Problem <a name="introduction"></a>

A buisness owner wishes to open a fast food resturaunt in the city of mumbai.A business man wishes to open a **chain of fast food restaurants** around the city of **Mumbai**. Mumbai being one of the busiest cities in the world also houses one of the busiest commuter rail systems with over 7.5 million commuters daily. 


He decides to look for the **ideal location which is close to a railway station with high footfall and in an area with a high density of other fast food restaurants.** This idea is based on the concept of game theory and Nesh equilibrium theorem that when shops of the same kind open close to one another tends to result in more profit.
He also wishes to acquire the price range (1 being the cheapest and 4 being the highest) and the ratings of the restaurants nearby.


This methodology is a single step in order to understand the best location for any establishment. Similar process can be used for schools, offices, hospitals, markets etc.
In this example only crowd from railways was considered. It could be made more precise by adding tourist attractions or other crowded areas.


## Data <a name="data"></a>

The neccessary data for the above problem will involve:

 - A list of all the fast food resturaunts in the area.
 - price range of these resturaunts
 - Rating
 - Location of the resturaunt (Latitude,Longitude)
 - A list of all the Railway station in the area.
 - Location of the station (Latitude, Longitude)

Data for the following command will be obtained mostly via foursquare:


 - 	Search command will be used to receive the venue ID of the restaurants.

 - 	Explore command will be used to obtain venue details (Name, Rating, cost, latitude, longitude)

 - 	Search command will be used once again to receive the details (Name, Latitude, Longitude) of the railway stations


https://nearfox.com/mumbai/10-busiest-stations-in-mumbai/ is to be data scraped to get the foot fall in the Top 10 busiest stations in Mumbai


## Functions used in the following program

In [1]:
import pandas as pd #For DataFrame Manipulation

import os #For accessing previously saved file

from geopy.geocoders import Nominatim #For accessing latitude and Longtitude

import folium #For Map Manipulation

import requests #For accessing url

import json #For json file manipulation

import numpy as np #Mathematical manipulation

from sklearn.cluster import KMeans #For clustering

import matplotlib.cm as cm #for plotting  on maps

import matplotlib.colors as colors #for color pallete

from math import radians, sin, cos, acos # To calculate the distance between 2 points based on Latitude and Longitude

from bs4 import BeautifulSoup # Data scraping

import requests # To get HTML file of web pages

### Foursquare API Details

In [2]:
CLIENT_ID = 'O4QXRWP31S2P4IFTE50GVWNNSH0MDF24YYFOIANX4RS55JGT' # your Foursquare ID
CLIENT_SECRET = '1Q31LNOALLLPX0TGLTDNOIBUQP0E3UJTRSSW5RVKPDSDWG4M' # your Foursquare Secret
VERSION = '20180323' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O4QXRWP31S2P4IFTE50GVWNNSH0MDF24YYFOIANX4RS55JGT
CLIENT_SECRET:1Q31LNOALLLPX0TGLTDNOIBUQP0E3UJTRSSW5RVKPDSDWG4M


The few code blocks below help us getting details neccesary in our project.

The search command gives us the venue id of the venues based on the following inputs:

https://api.foursquare.com/v2/venues/search?

 - near        : It is the name of the area, district or city of interest. Here we chosen Mumbai
 
 - Category ID : The category ID is used to specify the type of venues we are looking for,"4bf58dd8d48988d16e941735" is the  category id for fast food resturaunts
 
 - Radius      : Radius in which the venue should be present, we have chosen most of mumbai main area
 
 - limit       : Number of venues(100) 

In [3]:
LIMIT = 100
radius =10000
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&near=Mumbai&categoryId=4bf58dd8d48988d16e941735&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,  
    radius, 
    LIMIT)

In [4]:
results = requests.get(url).json()

In [5]:
len(results['response']['venues'])

50

In [6]:
Venue_ID=[]
for i in range(len(results['response']['venues'])):
    Venue_ID.append(results['response']['venues'][i]['id'])

In [7]:
Venue_ID[0:2]

['599567aa75a6ea6657f5515e', '4b7fd707f964a520374030e3']

Using the venue ID's collected in the previous api call, we will be getting the details of each venue using the venues call:
'https://api.foursquare.com/v2/venues/VENUE_ID'
<br> 


In [8]:
Details=[]

for i,v in enumerate(Venue_ID):
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
        v,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION)
    
    Details.append(requests.get(url).json())

len(Details)

50

From the above api call,the details of 'Name', 'Rating', 'Price', 'Latitude', 'Longitude' is taken and made into a dataframe
**DetailsPD**

In [9]:
Names=[]
Rating=[]
price=[]
Latitude=[]
Longitude=[]


for i in range(len(Details)):   
    Names.append(Details[i]['response']['venue']['name'])
    try:
        Rating.append(Details[i]['response']['venue']['rating'])
    except:
        Rating.append(np.nan)
    price.append(Details[i]['response']['venue']['price']['tier'])  
    Latitude.append(Details[i]['response']['venue']['location']['lat'])
    Longitude.append(Details[i]['response']['venue']['location']['lng'])

In [10]:
DetailsPD = pd.DataFrame(
    {'Names': Names,
     'Rating': Rating,
     'Price': price,
     'Latitude': Latitude,
     'Longitude': Longitude
    })

In [11]:
DetailsPD.head()

,Names,Rating,Price,Latitude,Longitude
0,McDonald's,6.6,1,19.072822,72.866768
1,McDonald's,7.5,1,19.141247,72.825579
2,McDonald's,6.8,1,19.165871,72.937905
3,KFC,6.3,1,19.173913,72.860192
4,McDonald's,6.5,1,19.065785,72.834304


In [12]:
DetailsPD['Rating'].fillna(round((DetailsPD['Rating'].mean())), inplace=True) #NA is converted to mean in 'Rating'

In [13]:
latitude=19.0760
longitude=72.8777  #Mumbai lat long

In [14]:
mumbai_map = folium.Map(location = [latitude, longitude], zoom_start = 12) #Declaring the map

In [15]:
#Map for marking all the fast food joints collected

mark = folium.map.FeatureGroup()

for lat, lng, in zip(DetailsPD.Latitude, DetailsPD.Longitude):
    mark.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, 
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
mumbai_map.add_child(mark)

In [16]:
X=DetailsPD.loc[:,['Latitude','Longitude']] #Getting Lat Long

## Clustering

In [17]:
#Clustering

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 3, 0, 4, 1, 4, 2, 4, 3])

In [18]:
# add clustering labels
X.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DetailsPD_merged = X.merge(DetailsPD, on=['Latitude','Longitude'])

DetailsPD_merged.head()

,Cluster Labels,Latitude,Longitude,Names,Rating,Price
0,2,19.072822,72.866768,McDonald's,6.6,1
1,0,19.141247,72.825579,McDonald's,7.5,1
2,3,19.165871,72.937905,McDonald's,6.8,1
3,0,19.173913,72.860192,KFC,6.3,1
4,4,19.065785,72.834304,McDonald's,6.5,1


In [19]:
print(round(DetailsPD_merged.Rating.mean(),2))
print(round(DetailsPD_merged.Price.mean(),2))

7.0
1.14


In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DetailsPD_merged['Latitude'], DetailsPD_merged['Longitude'], DetailsPD_merged['Names'], DetailsPD_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

### Getting Location details for railway stations

Same as above, here we are using the search function with similar inputs, with only difference being the category ID which is
'4bf58dd8d48988d129951735' which represents the train stations.

In [21]:
Rails=[]
VERSION='20180323'
for i,v in enumerate(Venue_ID):
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId=4bf58dd8d48988d129951735&near=Mumbai&radius=25000'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION)
    
    Rails.append(requests.get(url).json())

Creating a dataframe RailsPD with columns 'RNames','Rlatitude', 'Rlongitude'.

In [22]:
RNames=[]
RLatitude=[]
RLongitude=[]



for j in range(len(Rails[0]['response']['venues'])):
    RNames.append(Rails[0]['response']['venues'][j]['name']) 
    RLatitude.append(Rails[0]['response']['venues'][j]['location']['lat'])
    RLongitude.append(Rails[0]['response']['venues'][j]['location']['lng'])

In [23]:
RailsPD = pd.DataFrame(
    {'Names': RNames,
     'Latitude': RLatitude,
     'Longitude': RLongitude
    })

Since one of the clusters was heavily focused around a station which was not present in the foursquare venues we will be adding it manually

In [24]:
RailsPD.loc[30] = ['Goregaon railway station',19.1646,72.8493]

In [25]:
RailsPD.head()

,Names,Latitude,Longitude
0,Kurla Railway Station,19.065333,72.879335
1,Chhatrapati Shivaji Maharaj Terminus,18.940297,72.835384
2,Churchgate Railway Station,18.934706,72.827329
3,Wadala Railway Station,19.016792,72.858589
4,Thane Railway Station,19.186344,72.976082


dropping the stations which are nowhere close to any of the fast food venues

In [26]:
RailsPD = RailsPD.drop(RailsPD[(RailsPD.Latitude < 19.007698) | (RailsPD.Latitude > 19.164600) | (RailsPD.Longitude > 72.927984)].index)

In [27]:
RailsPD.reset_index(inplace=True,drop=True)
RailsPD.head()

,Names,Latitude,Longitude
0,Kurla Railway Station,19.065333,72.879335
1,Wadala Railway Station,19.016792,72.858589
2,Marol Naka Metro Station,19.108143,72.879373
3,Bandra Terminus,19.063308,72.841217
4,Dadar Railway Station,19.018678,72.843214


In [28]:
Rmap = folium.Map(location=[latitude, longitude], zoom_start=11)
Z = folium.map.FeatureGroup()
# loop through the stations names
for lat, lng, in zip(RailsPD.Latitude, RailsPD.Longitude):
    Z.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='black',
            fill_opacity=0.6
        )
    )

# add incidents to map
Rmap.add_child(Z)

The below code calculates the distance between 2 points using their Latitude and Longitude

In [29]:
def dist(Rlat,Rlon,Clat,Clon):
    
    Rlat = radians(Rlat)
    Rlon = radians(Rlon)
    Clat = radians(Clat)
    Clon = radians(Clon)
    
    dist = 6371.01 * acos(sin(Rlat)*sin(Clat) + cos(Rlat)*cos(Clat)*cos(Rlon - Clon))
    return dist

In [30]:
KClusterCent=pd.DataFrame(list(kmeans.cluster_centers_))

In [31]:
KClusterCent # Centres of clusters

,0,1
0,19.141203,72.843745
1,19.010711,72.837573
2,19.086701,72.886534
3,19.138130,72.923998
4,19.073229,72.836546


The following code gives all the stations which are within **2km** radius of the cluster centre 

In [32]:
C2C=[]
CNo=[]
for i in range(len(kmeans.cluster_centers_)):
    for j in range(len(RailsPD)):
        x=dist(RailsPD.iloc[j,1],RailsPD.iloc[j,2],KClusterCent.iloc[i,0],KClusterCent.iloc[i,1])
        
        if x<2:
            C2C.append(RailsPD.iloc[j,0])
            CNo.append(i)
            print("The station {} is close to cluster {} with a distance of {} km".format(RailsPD.iloc[j,0],i,round(x,3)))

The station Jogeshwari Railway Station is close to cluster 0 with a distance of 0.743 km
The station Dadar Railway Station is close to cluster 1 with a distance of 1.066 km
The station Elphinstone Road Railway Station is close to cluster 1 with a distance of 0.388 km
The station Vidyavihar Railway Station is close to cluster 2 with a distance of 1.397 km
The station Bandra Terminus is close to cluster 4 with a distance of 1.207 km
The station Santacruz Railway Station is close to cluster 4 with a distance of 1.054 km


We will be data scraping the **top 10 busiest Railway stations in mumbai** with their Foot fall

In [33]:
website_url = requests.get("https://nearfox.com/mumbai/10-busiest-stations-in-mumbai/").text
busy=BeautifulSoup(website_url,'lxml')
busy10=busy.find('div',{'class':'entry-content'})

In [34]:
Heading=list(map(str,busy10.findAll('h2')))

In [35]:
for i in range(len(Heading)):
    Heading[i]=Heading[i][4:-5] #To remove <h2></h2>

In [36]:
Heading.remove("Kalyan Station") #Kalyan station didnt follow the pattern while we underwent data scraping and as it was no where within 2 km of fast food resturaunts we decided to drop it.

In [37]:
strB10=str(busy10)

In [38]:
No=[]
indexN=[]
n="lakh"
for i,_ in enumerate(strB10):
    if strB10[i:i+len(n)]=="lakh":
        indexN.append(i)
    
    
for i,j in enumerate(indexN):
    No.append(strB10[j-6:j])

## Top 10 station with highest footfall

In [39]:
footfall=pd.DataFrame({'Station':Heading,'Footfall in Lakhs':No})
footfall

,Station,Footfall in Lakhs
0,Thane Station,6.54
1,Chhatrapati Shivaji Terminus Station (CST),6.36
2,Andheri Station,6.04
3,Dadar Station,5.77
4,Churchgate Station,5.05
5,Bandra Station,4.91
6,Virar Station,3.95
7,Kurla Station,3.81
8,Nallasopara Station,3.25


In [40]:
C2C=pd.DataFrame({'Station':C2C,'Cluster No':CNo})


C2C

,Station,Cluster No
0,Jogeshwari Railway Station,0
1,Dadar Railway Station,1
2,Elphinstone Road Railway Station,1
3,Vidyavihar Railway Station,2
4,Bandra Terminus,4
5,Santacruz Railway Station,4


In [41]:
A=C2C.merge(RailsPD,how='inner',left_on='Station',right_on='Names')

In [42]:
A.drop('Station',axis=1,inplace= True)
A

,Cluster No,Names,Latitude,Longitude
0,0,Jogeshwari Railway Station,19.136533,72.848797
1,1,Dadar Railway Station,19.018678,72.843214
2,1,Elphinstone Road Railway Station,19.007698,72.835719
3,2,Vidyavihar Railway Station,19.079478,72.897409
4,4,Bandra Terminus,19.063308,72.841217
5,4,Santacruz Railway Station,19.081106,72.842123


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon in zip(KClusterCent[0], KClusterCent[1]):
    folium.Circle(
        [lat, lon],
        radius=2000,
        color=rainbow[0],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.2).add_to(map_clusters)

for lat, lon,lab in zip(A.Latitude, A.Longitude,A.Names):
    label = folium.Popup(str(lab), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        popup=label,
        radius=5,
        color='yellow',
        fill=True,
        fill_color='purple',
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

## Results and Discussion

In our analysis it is seen that most of the fast food resturaunts is around 5 different areas and out of these 5 areas 2 of the areas have 2 different stations in a 2 km radius from the cluster radius. the 4 stations are as follows:

 - Elphinstone Road Railway Station
 - Dadar Railway Station
 - Bandra Terminus
 - Santacruz Railway Station
 
 
 Out of the 4 stations Dadar and Bandra are considered in the top 10 footfall list with Dadar having a footfall of approx  lakhs per day and Bandra having  lakhs per day
 
The competeitors in this area have a relitively cheap options and a rating of around 7

Our advice to the buisness owner would be to open either near Bandra or Dadar, having cheap options in order to get a decent rating. 


## Conclusion

The purpose of this project was to identify the area with the highest density of fast food resturaunts and to analyse the potential of high foot fall due to rail services.

As mentioned this is a methodology can be applied using even more deciding factors such as tourist areas, schools, offices etc. and the a better idea of the competitors could be achieved if more information on footfall, services available etc could be obtained. This can be implemented for any type of eshtablishment as well.